# Claude를 활용한 분류: 보험 고객 지원 티켓 분류기

이 가이드에서는 보험 고객 지원 티켓을 10개 카테고리로 분류하는 고정확도 분류 시스템을 구축합니다. 프롬프트 엔지니어링, 검색 증강 생성(RAG), 그리고 Chain-of-Thought 추론을 결합하여 분류 정확도를 70%에서 95% 이상으로 점진적으로 향상시키는 방법을 배웁니다.

이 가이드를 마치면 복잡한 비즈니스 규칙을 처리하고, 제한된 학습 데이터로 작동하며, 설명 가능한 결과를 제공하는 분류 시스템을 설계하는 방법을 이해하게 됩니다.

## 사전 요구사항
- Python 3.11+ 기본 지식
- Anthropic API 키 ([여기서 발급](https://console.anthropic.com))
- VoyageAI API 키 (선택사항 - 임베딩은 미리 계산되어 있음)
- 분류 문제에 대한 이해

## 환경 설정

먼저 필요한 패키지들을 설치합니다.

- anthropic 
- voyageai
- pandas
- matplotlib
- sklearn
- numpy

환경변수에서 API 키를 로드하고 모델명을 설정합니다.

In [ ]:
%%capture
!pip install -U anthropic voyageai pandas numpy matplotlib scikit-learn

In [ ]:
# API 키 설정 및 클라이언트 생성
import os

# 환경 변수에서 API 키 로드 (또는 직접 입력)
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"
# os.environ["VOYAGE_API_KEY"] = "your-voyage-api-key-here"

import anthropic
client = anthropic.Anthropic()
MODEL = "claude-haiku-4-5-20251001"

# 연결 테스트
response = client.messages.create(
    model=MODEL,
    max_tokens=10,
    messages=[{"role": "user", "content": "Hi"}]
)
print("✅ API 연결 성공:", response.content[0].text)
print(f"모델: {MODEL}")

## Claude를 활용한 분류

대규모 언어 모델(LLM)은 특히 기존 머신러닝 시스템이 어려움을 겪던 분야에서 분류 영역을 혁신했습니다. LLM은 복잡한 비즈니스 규칙이 있는 분류 문제와 저품질 또는 제한된 학습 데이터 시나리오에서 놀라운 성공을 보여주었습니다.

이 가이드에서는 LLM을 활용하여 고급 분류 작업을 수행하는 방법을 살펴봅니다:

1. **데이터 준비**: 학습 데이터와 테스트 데이터를 준비합니다.
2. **프롬프트 엔지니어링**: 분류를 위한 프롬프트 템플릿을 설계합니다.
3. **검색 증강 생성(RAG) 구현**: 벡터 데이터베이스를 사용하여 관련 예시를 검색합니다.
4. **테스트 및 평가**: 분류 시스템의 성능을 평가합니다.

## 문제 정의: 보험 고객 지원 티켓 분류기

*참고: 이 예제에서 사용된 문제 정의, 데이터, 레이블은 Claude 3 Opus가 합성 생성한 것입니다*

보험 산업에서 고객 지원은 고객 만족과 유지를 보장하는 데 핵심적인 역할을 합니다. 보험 회사는 매일 청구, 정책 관리, 청구 지원 등 다양한 주제를 다루는 대량의 지원 티켓을 받습니다.

#### 카테고리 정의

1. **Billing Inquiries** (청구 문의) - 청구서, 요금, 수수료, 보험료에 대한 질문
2. **Policy Administration** (정책 관리) - 정책 변경, 업데이트 또는 취소 요청
3. **Claims Assistance** (청구 지원) - 청구 절차 및 제출 방법에 대한 질문
4. **Coverage Explanations** (보장 범위 설명) - 보장되는 내용에 대한 질문
5. **Quotes and Proposals** (견적 및 제안) - 새 정책 견적 요청
6. **Account Management** (계정 관리) - 로그인 자격 증명 또는 비밀번호 재설정
7. **Billing Disputes** (청구 분쟁) - 예상치 못한 또는 잘못된 청구에 대한 불만
8. **Claims Disputes** (청구 이의) - 거부되거나 과소 지급된 청구에 대한 불만
9. **Policy Comparisons** (정책 비교) - 정책 옵션 간의 차이점에 대한 질문
10. **General Inquiries** (일반 문의) - 다른 카테고리에 맞지 않는 문의

## 데이터 로딩 및 준비

문제와 카테고리를 정의했으니, 레이블이 지정된 학습 데이터와 테스트 데이터를 로드합니다.

- 학습 데이터: 68개의 레이블된 예시 (RAG에 사용)
- 테스트 데이터: 68개의 예시 (평가에 사용)

In [ ]:
import pandas as pd
import textwrap

# 학습 데이터 로드
train_df = pd.read_csv('./data/train.tsv', sep='\t')
test_df = pd.read_csv('./data/test.tsv', sep='\t')

# 테스트 데이터 준비
X_test = test_df['text'].tolist()
y_test = test_df['label'].tolist()

# 레이블 목록
labels = [
    'Billing Inquiries',
    'Policy Administration', 
    'Claims Assistance',
    'Coverage Explanations',
    'Quotes and Proposals',
    'Account Management',
    'Billing Disputes',
    'Claims Disputes',
    'Policy Comparisons',
    'General Inquiries'
]

print(f"학습 데이터: {len(train_df)}개")
print(f"테스트 데이터: {len(test_df)}개")
print(f"카테고리: {len(labels)}개")

In [ ]:
# 카테고리 XML 정의
categories = """<category>
    <label>Billing Inquiries</label>
    <content>청구서, 요금, 수수료, 보험료에 대한 질문. 청구 명세서에 대한 설명 요청. 결제 방법 및 납부 기한에 대한 문의.</content>
</category>
<category>
    <label>Policy Administration</label>
    <content>정책 변경, 업데이트 또는 취소 요청. 정책 갱신 및 복원에 대한 질문. 보장 옵션 추가 또는 제거에 대한 문의.</content>
</category>
<category>
    <label>Claims Assistance</label>
    <content>청구 절차 및 제출 방법에 대한 질문. 청구 문서 제출에 대한 도움 요청. 청구 상태 및 지급 일정에 대한 문의.</content>
</category>
<category>
    <label>Coverage Explanations</label>
    <content>특정 정책 유형에서 보장되는 내용에 대한 질문. 보장 한도 및 제외 사항에 대한 설명 요청. 공제액 및 본인 부담 비용에 대한 문의.</content>
</category>
<category>
    <label>Quotes and Proposals</label>
    <content>새 정책 견적 및 가격 비교 요청. 이용 가능한 할인 및 번들 옵션에 대한 질문. 다른 보험사에서 전환하는 것에 대한 문의.</content>
</category>
<category>
    <label>Account Management</label>
    <content>로그인 자격 증명 또는 비밀번호 재설정 요청. 온라인 계정 기능에 대한 질문. 연락처 또는 개인 정보 업데이트에 대한 문의.</content>
</category>
<category>
    <label>Billing Disputes</label>
    <content>예상치 못한 또는 잘못된 청구에 대한 불만. 환불 또는 보험료 조정 요청. 연체료 또는 추심 통지에 대한 문의.</content>
</category>
<category>
    <label>Claims Disputes</label>
    <content>거부되거나 과소 지급된 청구에 대한 불만. 청구 결정 재고 요청. 청구 결과에 대한 이의 제기 문의.</content>
</category>
<category>
    <label>Policy Comparisons</label>
    <content>정책 옵션 간의 차이점에 대한 질문. 보장 수준 선택에 대한 도움 요청. 경쟁사 상품과의 비교에 대한 문의.</content>
</category>
<category>
    <label>General Inquiries</label>
    <content>회사 연락처 정보 또는 영업시간에 대한 질문. 제품 또는 서비스에 대한 일반적인 정보 요청. 다른 카테고리에 명확히 맞지 않는 문의.</content>
</category>"""

print("카테고리 정의 완료")

## 평가 프레임워크

분류기를 구축하기 전에 성능을 측정하기 위한 평가 프레임워크를 설정합니다.

1. **`evaluate()`**: 모든 테스트 예시에 대해 분류기를 실행하고 정확도를 계산합니다.
2. **`plot_confusion_matrix()`**: 혼동 행렬을 시각화하여 어떤 카테고리들이 혼동되는지 보여줍니다.

**Rate Limits**: `MAXIMUM_CONCURRENT_REQUESTS`는 기본적으로 1로 설정되어 있습니다. 더 높은 rate limit 티어가 있다면 이 값을 증가시켜 평가 속도를 높일 수 있습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 동시 요청 수 설정 (Rate limit에 따라 조정)
MAXIMUM_CONCURRENT_REQUESTS = 1

def evaluate(X, y, classifier, batch_size=MAXIMUM_CONCURRENT_REQUESTS):
    """분류기를 평가하고 정확도와 혼동 행렬을 반환합니다."""
    predictions = []
    
    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        futures = {executor.submit(classifier, text): i for i, text in enumerate(X)}
        results = [None] * len(X)
        
        for future in tqdm(as_completed(futures), total=len(X), desc="분류 중"):
            idx = futures[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                print(f"오류 발생: {e}")
                results[idx] = "General Inquiries"  # 기본값
        
        predictions = results
    
    # 정확도 계산
    accuracy = accuracy_score(y, predictions)
    print(f"\n정확도: {accuracy:.2%}")
    
    # 혼동 행렬 생성
    cm = confusion_matrix(y, predictions, labels=labels)
    
    return accuracy, cm, predictions

def plot_confusion_matrix(cm, labels):
    """혼동 행렬을 시각화합니다."""
    fig, ax = plt.subplots(figsize=(12, 10))
    
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    
    # 축 레이블 설정
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=labels, yticklabels=labels,
           title='혼동 행렬 (Confusion Matrix)',
           ylabel='실제 레이블',
           xlabel='예측 레이블')
    
    # x축 레이블 회전
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # 각 셀에 값 표시
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    fig.tight_layout()
    plt.show()

print("평가 프레임워크 준비 완료")

## 기준선: 랜덤 분류기

성능 기준선을 설정하기 위해 무작위로 카테고리를 선택하는 랜덤 분류기로 시작합니다.

10개 카테고리에서 무작위 추측은 약 **10%의 정확도**를 달성해야 합니다. 이것이 우리의 하한선입니다.

In [ ]:
import random

def random_classifier(text):
    """무작위로 카테고리를 선택하는 기준선 분류기"""
    return random.choice(labels)

# 랜덤 분류기 평가
print("=== 랜덤 분류기 평가 ===")
accuracy, cm, _ = evaluate(X_test, y_test, random_classifier)
plot_confusion_matrix(cm, labels)

무작위 추측에서 예상한 대로, 혼동 행렬은 모든 카테고리에 걸쳐 예측이 무의미한 패턴 없이 흩어져 있음을 보여줍니다.

이는 약 **10%의 정확도**라는 기준선을 확인합니다.

## 간단한 분류기 구축하기

이제 Claude를 사용하여 첫 번째 실제 분류기를 만들어봅시다. 세 가지 핵심 프롬프트 엔지니어링 기법을 사용합니다:

1. **구조화된 프롬프트 템플릿**: 카테고리와 티켓을 XML 형식으로 제공
2. **프리필링을 통한 출력 제어**: `<category>` 태그로 시작하여 레이블만 출력하도록 강제
3. **결정론적 분류**: `temperature=0.0`으로 일관된 예측 보장

In [ ]:
def simple_classify(X):
    """카테고리 정의만 사용하는 간단한 분류기"""
    prompt = textwrap.dedent(f"""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {categories}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {X}
    </ticket>

    Respond with just the label of the category between category tags.
    """)
    
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=100,
        temperature=0.0,
        model=MODEL,
    )
    
    return response.content[0].text.strip()

# 간단한 분류기 테스트
test_text = "I have a question about my recent bill. The amount seems higher than usual."
result = simple_classify(test_text)
print(f"테스트 입력: {test_text}")
print(f"분류 결과: {result}")

In [ ]:
# 간단한 분류기 평가
print("=== 간단한 분류기 평가 ===")
accuracy_simple, cm_simple, _ = evaluate(X_test, y_test, simple_classify)
plot_confusion_matrix(cm_simple, labels)

훨씬 나아졌습니다! **~70%의 정확도**로 무작위 추측보다 훨씬 뛰어납니다.

하지만 유사한 카테고리 간의 혼동이 있습니다. 이를 해결하기 위해 **검색 증강 생성(RAG)**을 추가합니다.

## VectorDB 클래스 정의

RAG를 구현하기 위해 벡터 데이터베이스 클래스를 정의합니다. VoyageAI의 임베딩 모델을 사용하여 유사한 예시를 검색합니다.

**참고**: 캐시된 임베딩이 `./data/vector_db.pkl`에 저장되어 있으므로 VoyageAI API 호출 없이도 작동합니다.

In [ ]:
import pickle
import json

class VectorDB:
    """간단한 벡터 데이터베이스 구현"""
    
    def __init__(self, api_key=None):
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = "./data/vector_db.pkl"
        self.client = None
        
        # VoyageAI 클라이언트 초기화 (선택사항)
        try:
            import voyageai
            if api_key is None:
                api_key = os.getenv("VOYAGE_API_KEY")
            if api_key:
                self.client = voyageai.Client(api_key=api_key)
        except ImportError:
            pass
    
    def load_db(self):
        """저장된 벡터 데이터베이스 로드"""
        if not os.path.exists(self.db_path):
            raise ValueError(f"벡터 데이터베이스 파일을 찾을 수 없습니다: {self.db_path}")
        
        with open(self.db_path, 'rb') as f:
            data = pickle.load(f)
        
        self.embeddings = data['embeddings']
        self.metadata = data['metadata']
        self.query_cache = json.loads(data.get('query_cache', '{}'))
        print(f"벡터 데이터베이스 로드 완료: {len(self.embeddings)}개 임베딩")
    
    def search(self, query, k=5, similarity_threshold=0.0):
        """쿼리와 가장 유사한 k개의 예시 검색"""
        # 쿼리 임베딩 가져오기 (캐시 또는 API 호출)
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        elif self.client:
            query_embedding = self.client.embed([query], model="voyage-2").embeddings[0]
            self.query_cache[query] = query_embedding
        else:
            # API 없이는 첫 번째 k개 예시 반환
            return [{'metadata': m, 'similarity': 0.0} for m in self.metadata[:k]]
        
        # 코사인 유사도 계산
        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1][:k]
        
        results = []
        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                results.append({
                    'metadata': self.metadata[idx],
                    'similarity': similarities[idx]
                })
        
        return results

# VectorDB 인스턴스 생성 및 로드
vectordb = VectorDB()
vectordb.load_db()

## RAG 강화 분류기

이제 검색 증강 생성을 적용한 분류기를 구축합니다:

1. **유사한 예시 검색**: 벡터 데이터베이스에서 5개의 유사한 학습 예시 검색
2. **Few-shot 예시 형식화**: XML 형식으로 예시를 구조화
3. **프롬프트 보강**: 예시를 포함하여 Claude가 더 나은 판단을 하도록 유도

In [ ]:
def rag_classify(X):
    """RAG를 사용하는 분류기"""
    # 유사한 예시 검색
    rag_examples = vectordb.search(X, k=5)
    
    # 예시를 XML 형식으로 구성
    rag_string = ""
    for example in rag_examples:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>"{example['metadata']['text']}"</query>
            <label>{example['metadata']['label']}</label>
        </example>
        """)
    
    prompt = textwrap.dedent(f"""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {categories}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {X}
    </ticket>

    Use the following examples to help you classify the query:
    <examples>
        {rag_string}
    </examples>

    Respond with just the label of the category between category tags.
    """)
    
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=100,
        temperature=0.0,
        model=MODEL,
    )
    
    return response.content[0].text.strip()

# RAG 분류기 테스트
test_text = "I have a question about my recent bill. The amount seems higher than usual."
result = rag_classify(test_text)
print(f"테스트 입력: {test_text}")
print(f"RAG 분류 결과: {result}")

In [ ]:
# RAG 분류기 평가
print("=== RAG 분류기 평가 ===")
accuracy_rag, cm_rag, _ = evaluate(X_test, y_test, rag_classify)
plot_confusion_matrix(cm_rag, labels)

## RAG 결과 분석

RAG가 정확도를 ~70%에서 **~94%**로 향상시켰습니다! 혼동 행렬은 훨씬 더 강한 대각선을 보여줍니다.

남은 오류를 해결하기 위해 **Chain-of-Thought 추론**을 추가합니다.

## RAG와 Chain-of-Thought 추론

Chain-of-Thought (CoT) 프롬프팅은 Claude에게 분류 결정을 내리기 전에 "소리 내어 생각"하도록 요청합니다.

이 명시적인 추론 과정은 "Billing Inquiry"와 "Billing Dispute"처럼 미묘한 차이를 구분하는 데 도움이 됩니다.

In [ ]:
def rag_chain_of_thought_classify(X):
    """RAG + Chain-of-Thought를 사용하는 분류기"""
    # 유사한 예시 검색
    rag_examples = vectordb.search(X, k=5)
    
    # 예시를 XML 형식으로 구성
    rag_string = ""
    for example in rag_examples:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>"{example['metadata']['text']}"</query>
            <label>{example['metadata']['label']}</label>
        </example>
        """)
    
    prompt = textwrap.dedent(f"""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {categories}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {X}
    </ticket>

    Use the following examples to help you classify the query:
    <examples>
        {rag_string}
    </examples>

    First you will think step-by-step about the problem in scratchpad tags.
    You should consider all the information provided and create a concrete argument for your classification.

    Respond using this format:
    <response>
        <scratchpad>Your thoughts and analysis go here</scratchpad>
        <category>The category label you chose goes here</category>
    </response>
    """)
    
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<response><scratchpad>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=1000,
        temperature=0.0,
        model=MODEL,
    )
    
    # 카테고리 추출
    result = response.content[0].text
    if "<category>" in result:
        return result.split("<category>")[1].strip()
    return result.strip()

# RAG + CoT 분류기 테스트
test_text = "I have a question about my recent bill. The amount seems higher than usual."
result = rag_chain_of_thought_classify(test_text)
print(f"테스트 입력: {test_text}")
print(f"RAG + CoT 분류 결과: {result}")

In [ ]:
# RAG + Chain-of-Thought 분류기 평가
print("=== RAG + Chain-of-Thought 분류기 평가 ===")
accuracy_cot, cm_cot, _ = evaluate(X_test, y_test, rag_chain_of_thought_classify)
plot_confusion_matrix(cm_cot, labels)

## 결과 요약

Chain-of-Thought 추론이 정확도를 **~97%**로 끌어올렸습니다!

### 점진적 개선 요약

| 접근 방식 | 정확도 |
|----------|--------|
| 랜덤 기준선 | ~10% |
| 간단한 분류기 | ~70% |
| RAG 분류기 | ~94% |
| **RAG + Chain-of-Thought** | **~97%** |

In [ ]:
# 최종 결과 비교 시각화
methods = ['랜덤 기준선', '간단한 분류기', 'RAG 분류기', 'RAG + CoT']
accuracies = [0.10, accuracy_simple, accuracy_rag, accuracy_cot]

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, accuracies, color=['gray', 'steelblue', 'green', 'darkgreen'])
plt.ylabel('정확도')
plt.title('분류 접근 방식별 정확도 비교')
plt.ylim(0, 1.0)

# 막대 위에 값 표시
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{acc:.1%}', ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()

## 다음 단계

이 가이드에서 배운 내용:
1. **프롬프트 엔지니어링**: XML 형식, 프리필링, temperature 설정
2. **RAG**: 유사한 예시를 검색하여 few-shot 학습 구현
3. **Chain-of-Thought**: 명시적 추론으로 정확도 향상

### 추가 개선 방향
- **Promptfoo로 대규모 평가**: `./evaluation/` 폴더의 설정 참조
- **더 많은 학습 데이터**: RAG의 효과를 높이기 위해 예시 추가
- **파인튜닝된 임베딩**: 도메인 특화 임베딩 모델 사용
- **앙상블 방법**: 여러 분류기 결과 조합